In [40]:
from sqlalchemy import create_engine
import random
import pandas as pd
import numpy as np
import names
import string

### Ufuncs

def _clamp(x):
    return max(0, min(x, 255))


def col_gen():
    return "#{0:02x}{1:02x}{2:02x}".format(
        _clamp(random.randint(10, 255)),
        _clamp(random.randint(10, 255)),
        _clamp(random.randint(10, 255)),
    )

## Donut chart

* Two tables: data and fill
* In data table it will have two columns: label and value
* In fill table it will have two columns: label and color
* As next step we need to have a drop down menu which will filter the data which has to be shown in the chart and in the backend we will send a list of key value pair where value will be of the same format as it is for a single selection of the drop down menu.


* __Reference:__ https://codesandbox.io/s/6hr9o

In [41]:
### Generate data

label_len = 10
label = [names.get_full_name() for _ in range(label_len)]
value = [random.randint(1, 100) for _ in range(label_len)]
fill = [col_gen() for _ in range(label_len)]
donut_df = pd.DataFrame({'label': label, 'value': value})
label_fill_df = pd.DataFrame({'label': label, 'fill': fill})

### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
donut_df.to_sql('donut_data_table', disk_engine, if_exists='append')
label_fill_df.to_sql('donut_fill_table', disk_engine, if_exists='append')

### Get data

data_df = pd.read_sql_query('SELECT * FROM donut_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM donut_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


### Parse data for chart

response_payload = ((data_df.set_index('label')).join(fill_df.set_index('label'))).reset_index().to_dict('records')
response_payload

[{'label': 'Joseph Moran', 'value': 76, 'fill': '#214fec'},
 {'label': 'Gregory Mancini', 'value': 6, 'fill': '#af9093'},
 {'label': 'Kurt Hashimoto', 'value': 30, 'fill': '#d6ff15'},
 {'label': 'Keneth Manning', 'value': 6, 'fill': '#fae0c8'},
 {'label': 'Dorothy Anderson', 'value': 69, 'fill': '#35b2bd'},
 {'label': 'Adele Thompson', 'value': 63, 'fill': '#279c48'},
 {'label': 'Edith Bucio', 'value': 28, 'fill': '#3a98fa'},
 {'label': 'Larry Taylor', 'value': 82, 'fill': '#677ed9'},
 {'label': 'Betty Walker', 'value': 36, 'fill': '#a9788b'},
 {'label': 'Marla Stafford', 'value': 25, 'fill': '#1564d7'}]

# Stacked Bar Chart


* __Reference:__ https://www.geeksforgeeks.org/create-a-stacked-bar-chart-using-recharts-in-reactjs/

In [42]:
### Generate data
n_label = 6
n_var = 3
label = [names.get_full_name() for _ in range(n_label)]
var_collector = []
data_collector = []
cols_collector = []

while len(set(var_collector))< n_var:
    var_collector.append(random.choice(string.ascii_lowercase))
    data_collector.append(np.random.uniform(1,100,n_label))
    cols_collector.append(col_gen())

data_df = pd.DataFrame(data_collector, index =var_collector, columns = label).T.reset_index().rename(columns = {"index":"label"}).round(2)
fill_df = pd.DataFrame({'label': var_collector, 'fill': cols_collector})

### Save data

disk_engine = create_engine('sqlite:///charts_db.db')
data_df.to_sql('stackedchart_data_table', disk_engine, if_exists='append')
fill_df.to_sql('stackedchart_fill_table', disk_engine, if_exists='append')

### Get data

data_df = pd.read_sql_query('SELECT * FROM stackedchart_data_table',disk_engine)
data_df = data_df.drop('index',axis= 1)

fill_df = pd.read_sql_query('SELECT * FROM stackedchart_fill_table', disk_engine)
fill_df = fill_df.drop('index',axis= 1)


### Parse data for chart

response_payload = {"data":data_df.to_dict('records'),"fill":fill_df.to_dict('records')}
response_payload

{'data': [{'label': 'Scott Alfaro', 'd': 90.42, 'z': 47.02, 'h': 84.86},
  {'label': 'James Weber', 'd': 70.42, 'z': 29.18, 'h': 97.85},
  {'label': 'Mary Carson', 'd': 76.83, 'z': 9.09, 'h': 17.14},
  {'label': 'Johnny Herman', 'd': 31.47, 'z': 33.05, 'h': 27.51},
  {'label': 'Kerry Goodwin', 'd': 17.16, 'z': 78.58, 'h': 84.95},
  {'label': 'Mark Mertens', 'd': 48.38, 'z': 10.66, 'h': 77.51}],
 'fill': [{'label': 'd', 'fill': '#bd4280'},
  {'label': 'z', 'fill': '#45517b'},
  {'label': 'h', 'fill': '#546b53'}]}